# EXPLORATION_04. 작사가 인공지능 만들기

## (1). 데이터 다운로드

mkdir -p ~/aiffel/lyricist/models  
ln -s ~/data ~/aiffel/lyricist/data  
위에 코드를 Cloud shell 에 입력 후 데이터를 다운 받아 준비!!

## (2). 데이터 읽어오기

In [1]:
import glob
import os
import re
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

## raw_corpus 라는 빈 리스트를 만들었다. 
## 그 후에 txt 파일을 모두 읽어서 우리가 만들어 둔 리스트에 담는다!!!!

for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


우리가 원하는 데이터만 받아내고 필요없는 문장들은 빼고 보고싶다!  
1. 그래서 일단 아무말이 없는 문장을 빼고 싶기 때문에 len(sentence)가 0이면 건너뛰게 할 것이다. (sentence)의 길이!  
2. 그리고 다음으로는 문장의 끝이 ":" 이렇게 끝나면 화자 이름 : 이런느낌이니까 그런 문장도 건너뛰게 해준다.  
일단 우리가 필요없는 문장이 잘 빠졌나 10개만 추출해서 확인해보자!!

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   
    if sentence[-1] == ":": continue  

    if idx > 9: break   
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


## (3). 데이터 정제

문장을 일정한 기준으로 쪼갤 것이다. 이 과정을 토큰화라고 한다!!!  
사실 띄어쓰기를 기준으로 두는게 제일 간단한데 그 때 문제가 발생할 수 도 있다.

입력된 문장을  
(#1) => 소문자로 바꾼다음, 양쪽 공백을 지워준다.  
(#2) => 특수문자 양쪽에 공백을 하나 넣는다. 예를 들어 ? 나 ! 같은 거임  
(#3) => 여러개의 공백은 하나의 공백으로 바꿔준다.  
(#4) => a-zA-Z, ?, !, . , , 뭐 이런 애들이 아니면 하나의 공백으로 바꿔준다.  
(#5) => 다시 양쪽 공백을 지워준다.  
(#6) => 문장 시작에는 <start>, 끝에는 <end> 를 추가한다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

## 이 문장이 어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


너무 긴 문장은 노래 가사를 작사하는데 어울리지 않는다.  
=> 그래서 토큰 개수가 15개 넘어가지 않게 split() 으로 제한을 두었다.(밑에 코드!!)

In [4]:
## 여기에 빈 리스트를 만들어서 아까 위에서 정재 시킨 문장을 넣어준다.
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뛴다.
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(preprocess_sentence(sentence).split()) > 15: continue
    corpus.append(preprocess_sentence(sentence))
        
## 잘 정재되었는지 확인해보자! 너무 많으니까 10개만!
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## (4). 평가 데이터셋 분리

tokenize() 함수로 데이터를 Tensor 로 변환한 후에 , sklearn 모듈의 train_test_split() 함수를 사용해서 훈련데이터랑 평가 데이터를 분리해준다.  
(조건!!!!) => 단어장의 크기는 12,000이상이고 총 데이터의 20% 를 평가 데이터셋으로 사용

In [5]:
def tokenize(corpus):
    ## 12000단어를 기억할 수 있는 tokenizer를 만든다.
    ## 12000단어에 포함되지 못한 단어는 '<unk>'로 바꾼다.
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    ## corpus를 이용해 tokenizer 내부의 단어장을 완성한다.
    tokenizer.fit_on_texts(corpus)
    ## 준비한 tokenizer를 이용해 corpus를 Tensor로 변환한다.
    tensor = tokenizer.texts_to_sequences(corpus)   

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe61c262fa0>


In [6]:
## 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력해서 데이터 값 확인!

print(tensor[:3, :10])

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [7]:
## 텐서데이터는 전부 정수이다. 숫자는 아까 우리가 tokenizer에 구축된 단어 사전의 인덱스이다.
## 단어 사전이 어떻게 구축되었는지 확인해보기!!

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
## tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다.

src_input = tensor[:, :-1]  
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [9]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (124810, 14)
Target Train:  (124810, 14)


In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

## tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

## 준비한 데이터 소스로부터 데이터셋을 만든다.

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [11]:
## 만드는 모델에는 1개의 embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성
## 일단 지금은 구조도에 설명된 정도만 이해하고 pass~~~~

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

## embedding 레이어는 이 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.

## (5). 인공지능 만들기

In [12]:
## 데이터셋에서 데이터 한 배치만 불러오는 방법이다.

for src_sample, tgt_sample in dataset.take(1): break

## 한 배치만 불러온 데이터를 모델에 넣어보자!
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 6.9579597e-05,  7.9778139e-05, -8.0385063e-05, ...,
          1.5380612e-05,  1.4584203e-04, -7.1330542e-05],
        [-8.6160653e-05,  2.3694332e-05,  2.6869719e-05, ...,
         -7.4822819e-05, -8.8675142e-06, -1.5179208e-04],
        [-1.3851011e-04,  1.5108044e-04,  3.3125456e-04, ...,
         -2.9256468e-04,  7.3004419e-05,  3.7309415e-05],
        ...,
        [ 6.3403777e-04,  6.3120147e-05,  1.8700586e-03, ...,
          1.4739120e-03,  6.2014395e-04,  8.0716040e-04],
        [ 6.7187264e-04,  7.4461648e-05,  1.7239354e-03, ...,
          1.3620409e-03,  4.4154195e-04,  1.0111111e-03],
        [ 7.3472498e-04,  1.2781932e-05,  1.4848493e-03, ...,
          1.1072840e-03,  1.5929212e-04,  1.1940884e-03]],

       [[ 6.9579597e-05,  7.9778139e-05, -8.0385063e-05, ...,
          1.5380612e-05,  1.4584203e-04, -7.1330542e-05],
        [ 3.7509244e-04,  9.1152033e-05,  2.2766324e-04, ...,
          1.7290680e-04, 

In [13]:
## 어떻게 생겼는지 보자!

model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [14]:
## optimizer와 loss등은 차차 배우니까 일단 어떤느낌인지만 알고 넘기기~

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data = (enc_val, dec_val))

Epoch 1/10
488/488 [==============================] - 240s 487ms/step - loss: 3.4085 - val_loss: 3.0249
Epoch 2/10
488/488 [==============================] - 248s 508ms/step - loss: 2.8826 - val_loss: 2.8033
Epoch 3/10
488/488 [==============================] - 249s 510ms/step - loss: 2.6460 - val_loss: 2.6499
Epoch 4/10
488/488 [==============================] - 250s 512ms/step - loss: 2.4275 - val_loss: 2.5291
Epoch 5/10
488/488 [==============================] - 249s 510ms/step - loss: 2.2135 - val_loss: 2.4352
Epoch 6/10
488/488 [==============================] - 250s 512ms/step - loss: 2.0093 - val_loss: 2.3584
Epoch 7/10
488/488 [==============================] - 250s 512ms/step - loss: 1.8161 - val_loss: 2.3004
Epoch 8/10
488/488 [==============================] - 249s 510ms/step - loss: 1.6366 - val_loss: 2.2520
Epoch 9/10
488/488 [==============================] - 250s 512ms/step - loss: 1.4732 - val_loss: 2.2261
Epoch 10/10
488/488 [==============================] - 250s 512m

In [20]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    ## 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    ## 단어 하나씩 예측해 문장을 만든다.
    ##    1. 입력받은 문장의 텐서를 입력한다.
    ##    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아낸다.
    ##    3. 2에서 예측된 word index를 문장 뒤에 붙인다.
    ##    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마친다.
    while True:
        ## 1
        predict = model(test_tensor) 
        ## 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        ## 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        ## 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    ## tokenizer를 이용해 word index를 단어로 하나씩 변환한다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
## 문장 생성 함수를 실행해서 i love 뒤에 어떤 문장이 나오는지 보자!!

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you , i love you , <end> '

# 회고!

## <어려웠던 점>  
=> 일단....사실 너무 지루했다....어떤 프로젝트이든 똑같겠지만! 함수랑 용어가 많고 코드가 조금 긴 편이라서 오류를 잡을 때 시간이 많이 걸렸던 것 같다. lms 노드에서도 넘어간 부분은 이해가 안되서 시간이 걸렸던 것 같다. 또한, 학습하는데 시간이 너무 많이 걸려서 다양한 시도를 해보는게 어려웠던 것 같다. 

## <알아낸 점> & <아직까지 모호한 점>  
=> 제일 처음에 sequence 라는게 나오길래 학과에서 많이 쓰이는 수열이라고 생각을 햇는데 파이썬에서 시퀀스 자료형이라는게 있는걸 보고 신기했다. 정말 사소한 거였지만 내가 알고 있는 개념과 조금 달라서 잘 기록해뒀다. 

## <시도한 것들>  
=> 저번에 익스시간에 코드를 짜다가 중간에 그만뒀는데! 그 후에 다시 코드를 수정하려고 했는데 에러가 엄청 떠서 당황을 했다. 당황을 해서 이것저것 건들이니, 더 에러가 많이 생기고 결과가 이상해져서 아예 싹 다 지우고 다시 데이터 다듬기부터 시작했다. 그래서 시간이 많이 걸렸지만 에러가 나오지 않고 결과가 나왔다. 첨에는 val_loss값이 높아서 epochs을 10으로 설정하고 가능할까 생각을 했지만! hidden size를 1024에서 2048로 변경했더니 loss값이 학습을 할 때마다 눈에 확연하게 떨어졌다. 

## <자기 다짐>  
=> 마지막 i love you, 다음에 나오는 결과가 너무 귀엽게 나와서 밤에 늦게까지 한 보람이 있었던 것 같다. 제발....노드를 미루지 말고...미리하자....그래도 이번에는 미리 시작을 했지만! 노드를 미루니까 급하게 코드를 수정하기 애매하고 학습시간이 오래걸려서 애를 먹었던 것 같다! 미루지말자....제발....ㅎ
